In [1]:
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null  # Install Java 11
!wget -qO - https://archive.apache.org/dist/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz | tar xzf - -C /opt/  # Install Spark 3.5.5
!pip install findspark  # Install findspark to locate Spark


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,369 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,730 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,774 kB]
Get:13 http://archive.ubuntu.com

In [3]:
!export SPARK_HOME=/opt/spark-3.5.5-bin-hadoop3
!export PATH=$SPARK_HOME/bin:$PATH
!export PYTHONPATH=$SPARK_HOME/python:$PYTHONPATH


In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.5-bin-hadoop3"


In [8]:
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [9]:
!pip install pyspark


In [1]:
!pip install pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("PokemonFeistinessAnalysis") \
    .getOrCreate()

print("✅ Spark Initialized Successfully!")


✅ Spark Initialized Successfully!


In [3]:
from google.colab import files
uploaded = files.upload()


Saving archive (1).zip to archive (1).zip


In [4]:
!unzip "/content/archive (1).zip" -d "/content/pkeman dataset"


Archive:  /content/archive (1).zip
  inflating: /content/pkeman dataset/pokemon.csv  


In [5]:
import pandas as pd

# Read CSV using Pandas
df = pd.read_csv("/content/pkeman dataset/pokemon.csv")  # Change this if your filename is different

# Convert Pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(df)

# Show first few rows
spark_df.show(5)


+--------------------+-----------+------------+--------------+----------------+-------------+-------------+------------+--------------+-------------+-------------+--------------+-----------+--------------+--------------+---------------+------------+-------------+-------------+------+--------------+--------------+----------+------------+--------------+-------+-----------------+--------+---+---------------------+----------+---------------+--------------+---------+----------+-----+-----+------+---------+----------+------------+
|           abilities|against_bug|against_dark|against_dragon|against_electric|against_fairy|against_fight|against_fire|against_flying|against_ghost|against_grass|against_ground|against_ice|against_normal|against_poison|against_psychic|against_rock|against_steel|against_water|attack|base_egg_steps|base_happiness|base_total|capture_rate| classfication|defense|experience_growth|height_m| hp|        japanese_name|      name|percentage_male|pokedex_number|sp_attack|sp_

In [6]:
# Fix column names
for col_name in spark_df.columns:
    spark_df = spark_df.withColumnRenamed(col_name, col_name.strip().lower())


In [7]:
spark_df.printSchema()


root
 |-- abilities: string (nullable = true)
 |-- against_bug: double (nullable = true)
 |-- against_dark: double (nullable = true)
 |-- against_dragon: double (nullable = true)
 |-- against_electric: double (nullable = true)
 |-- against_fairy: double (nullable = true)
 |-- against_fight: double (nullable = true)
 |-- against_fire: double (nullable = true)
 |-- against_flying: double (nullable = true)
 |-- against_ghost: double (nullable = true)
 |-- against_grass: double (nullable = true)
 |-- against_ground: double (nullable = true)
 |-- against_ice: double (nullable = true)
 |-- against_normal: double (nullable = true)
 |-- against_poison: double (nullable = true)
 |-- against_psychic: double (nullable = true)
 |-- against_rock: double (nullable = true)
 |-- against_steel: double (nullable = true)
 |-- against_water: double (nullable = true)
 |-- attack: long (nullable = true)
 |-- base_egg_steps: long (nullable = true)
 |-- base_happiness: long (nullable = true)
 |-- base_total: 

In [8]:
from pyspark.sql.functions import col, round, desc, row_number
from pyspark.sql.window import Window

# Ensure weight_kg and attack are not null or zero
filtered_df = spark_df.filter(
    (col("weight_kg").isNotNull()) & (col("weight_kg") > 0) & (col("attack").isNotNull())
)

# Compute Feistiness
feistiness_df = filtered_df.withColumn(
    "feistiness", round(col("attack") / col("weight_kg"), 2)
)

# Find the feistiest Pokémon for each type1
windowSpec = Window.partitionBy("type1").orderBy(desc("feistiness"))

feistiest_pokemon = (
    feistiness_df.withColumn("rank", row_number().over(windowSpec))
    .filter(col("rank") == 1)
    .select("type1", "type2", "name", "feistiness")
)

# Show results
feistiest_pokemon.show()


+--------+--------+----------+----------+
|   type1|   type2|      name|feistiness|
+--------+--------+----------+----------+
|     bug|   fairy|  Cutiefly|     225.0|
|    dark|  flying|   Murkrow|     40.48|
|  dragon|     NaN|   Dratini|     19.39|
|electric|electric|    Raichu|       NaN|
|   fairy|     NaN|   Flabébé|     380.0|
|fighting|     NaN|Crabrawler|     11.71|
|    fire|     ice|    Vulpix|       NaN|
|  flying|  dragon|    Noibat|      3.75|
|   ghost|  poison|   Haunter|     500.0|
|   grass| psychic| Exeggutor|       NaN|
|  ground|     ice| Sandshrew|       NaN|
|     ice|     NaN| Vanillite|      8.77|
|  normal|    dark|   Rattata|       NaN|
|  poison|  poison|    Grimer|       NaN|
| psychic|   ghost|     Hoopa|       NaN|
|    rock|  ground|   Geodude|       NaN|
|   steel| psychic|   Jirachi|     90.91|
|   water|     NaN|Wishiwashi|    466.67|
+--------+--------+----------+----------+



In [9]:
from pyspark.sql.functions import col, round, desc, row_number
from pyspark.sql.window import Window

# Ensure weight_kg and attack are not null and weight_kg > 0
filtered_df = spark_df.filter(
    (col("weight_kg").isNotNull()) &
    (col("weight_kg") > 0) &
    (col("attack").isNotNull())
)

# Compute Feistiness (Attack / Weight)
feistiness_df = filtered_df.withColumn(
    "feistiness", round(col("attack") / col("weight_kg"), 2)
)

# Find the feistiest Pokémon per type
windowSpec = Window.partitionBy("type1").orderBy(desc("feistiness"))

feistiest_pokemon = (
    feistiness_df.withColumn("rank", row_number().over(windowSpec))
    .filter(col("rank") == 1)
    .select("type1", "type2", "name", "feistiness")
)

# Show results
feistiest_pokemon.show()


+--------+--------+----------+----------+
|   type1|   type2|      name|feistiness|
+--------+--------+----------+----------+
|     bug|   fairy|  Cutiefly|     225.0|
|    dark|  flying|   Murkrow|     40.48|
|  dragon|     NaN|   Dratini|     19.39|
|electric|electric|    Raichu|       NaN|
|   fairy|     NaN|   Flabébé|     380.0|
|fighting|     NaN|Crabrawler|     11.71|
|    fire|     ice|    Vulpix|       NaN|
|  flying|  dragon|    Noibat|      3.75|
|   ghost|  poison|   Haunter|     500.0|
|   grass| psychic| Exeggutor|       NaN|
|  ground|     ice| Sandshrew|       NaN|
|     ice|     NaN| Vanillite|      8.77|
|  normal|    dark|   Rattata|       NaN|
|  poison|  poison|    Grimer|       NaN|
| psychic|   ghost|     Hoopa|       NaN|
|    rock|  ground|   Geodude|       NaN|
|   steel| psychic|   Jirachi|     90.91|
|   water|     NaN|Wishiwashi|    466.67|
+--------+--------+----------+----------+



In [10]:
filtered_df = spark_df.filter(
    (col("weight_kg").isNotNull()) &
    (col("weight_kg") > 0) &
    (col("attack").isNotNull())
)

# Drop any remaining rows with null feistiness
filtered_df = filtered_df.na.drop(subset=["weight_kg", "attack"])

# Compute Feistiness
feistiness_df = filtered_df.withColumn(
    "feistiness", round(col("attack") / col("weight_kg"), 2)
)

# Find the feistiest Pokémon per type
windowSpec = Window.partitionBy("type1").orderBy(desc("feistiness"))

feistiest_pokemon = (
    feistiness_df.withColumn("rank", row_number().over(windowSpec))
    .filter(col("rank") == 1)
    .select("type1", "type2", "name", "feistiness")
)

# Show results
feistiest_pokemon.show()


+--------+-------+----------+----------+
|   type1|  type2|      name|feistiness|
+--------+-------+----------+----------+
|     bug|  fairy|  Cutiefly|     225.0|
|    dark| flying|   Murkrow|     40.48|
|  dragon|    NaN|   Dratini|     19.39|
|electric|  ghost|     Rotom|    216.67|
|   fairy|    NaN|   Flabébé|     380.0|
|fighting|    NaN|Crabrawler|     11.71|
|    fire|    NaN|   Torchic|      24.0|
|  flying| dragon|    Noibat|      3.75|
|   ghost| poison|   Haunter|     500.0|
|   grass|  steel|   Kartana|    1810.0|
|  ground|    NaN|   Drilbur|      10.0|
|     ice|    NaN| Vanillite|      8.77|
|  normal|    NaN|  Castform|      87.5|
|  poison|    NaN|   Koffing|      65.0|
| psychic|    NaN|     Azelf|    416.67|
|    rock|  fairy|   Diancie|     18.18|
|   steel|psychic|   Jirachi|     90.91|
|   water|    NaN|Wishiwashi|    466.67|
+--------+-------+----------+----------+



In [11]:
# Save results as CSV
feistiest_pokemon.write.csv("feistiest_pokemon_final.csv", header=True, mode="overwrite")

# Download the CSV file
from google.colab import files
files.download("feistiest_pokemon_final.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>